In [15]:
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report
from sklearn import tree

import graphviz
from graphviz import Graph

import prepare
import preprocessing

pd.set_option('display.max_columns', None)

In [2]:
df = prepare.get_data()
df

,sso_id,report_date,spill_address_num,spill_st_name,total_gal,gals_ret,spill_start,spill_stop,hrs,cause,comments,actions,watershed,unit_id,unit_id2,discharge_to,discharge_route,council_district,month,year,week,earz_zone,pipe_diam,pipe_len,pipe_type,inst_year,inches_no,rainfall_last3,spill_address_full,num_spills_recorded,num_spills_24mos,prevspill_24mos,unit_type,asset_type,last_cleaned,response_time,response_dttm,public_notice,root_cause,hrs_2,gal_2,hrs_3,gal_3,days_since_cleaned,total_gal_binned,country_address,location,zip_code,age,age_binned,time_spilled,precipitation,snowfall,snow_depth,avg_temp,max_temp,min_temp,foggy,heavy_fog,thunder,hail,blowing_snow,rain,precipitation_rolling_7,precipitation_rolling_14,precipitation_rolling_30,avg_temp_rolling_7,avg_temp_rolling_14,avg_temp_rolling_30,max_temp_rolling_7,max_temp_rolling_14,max_temp_rolling_30,min_temp_rolling_7,min_temp_rolling_14,min_temp_rolling_30
0,6582,2019-03-10,3200,THOUSAND OAKS DR,2100,2100.0,2019-03-10 13:16:00,2019-03-10 14:40:00,1.400000,Grease,Spill ContainedReturned to SystemArea Cleaned ...,CLEANED MAIN,SALADO CREEK,66918,66917,STREET,None,NaN,3,2019,11,0,8.0,16.550000,PVC,1997.0,NaN,NaN,3200 THOUSAND OAKS DR,1,1.0,0,GRAVITY,Sewer Main,NaN,27.0,2019-03-10 13:43:00,False,NaN,0.00,0.0,0.0,0.0,NaN,"(1000, 5000]","3200 THOUSAND OAKS DR,SAN ANTONIO,Texas,USA","3200, Thousand Oaks Drive, Horseshoe Bend, Los...",78746,22.0,"(20, 25]",01:24:00,0.00,0.0,0.0,69.0,79,60,1,1,0,0,0,0,4.285714e-03,0.005714,0.016333,54.285714,55.285714,54.200000,65.857143,65.714286,64.533333,46.000000,47.642857,45.900000
1,6583,2019-03-10,6804,S FLORES ST,80,0.0,2019-03-10 14:25:00,2019-03-10 15:45:00,1.333333,Grease,Spill ContainedArea Cleaned and Disinfected,CLEANED MAIN,DOS RIOS,24250,24193,STORMDRAIN,None,3.0,3,2019,11,0,8.0,157.000000,PVC,1988.0,NaN,NaN,6804 S FLORES,1,1.0,0,GRAVITY,Sewer Main,NaN,65.0,2019-03-10 15:30:00,False,NaN,0.00,0.0,0.0,0.0,NaN,"(50, 250]","6804 S FLORES,SAN ANTONIO,Texas,USA","Flores, West Odessa, Ector County, Texas, Unit...",Texas,31.0,"(30, 35]",01:20:00,0.00,0.0,0.0,69.0,79,60,1,1,0,0,0,0,4.285714e-03,0.005714,0.016333,54.285714,55.285714,54.200000,65.857143,65.714286,64.533333,46.000000,47.642857,45.900000
2,6581,2019-03-09,215,AUDREY ALENE DR,79,0.0,2019-03-09 18:00:00,2019-03-09 19:30:00,1.500000,Structural,Spill ContainedArea Cleaned and DisinfectedFlu...,CLEANED MAIN,DOS RIOS,2822,3351,ALLEY,None,1.0,3,2019,10,0,8.0,350.000000,CP,1955.0,NaN,NaN,215 Audrey Alene Dr,1,1.0,0,GRAVITY,Sewer Main,NaN,60.0,2019-03-09 19:00:00,False,NaN,1.15,69.0,0.0,0.0,NaN,"(50, 250]","215 Audrey Alene Dr,SAN ANTONIO,Texas,USA","215, Audrey Alene Drive, San Antonio, Bexar Co...",78216,64.0,"(60, 65]",01:30:00,0.03,0.0,0.0,74.0,87,60,1,0,0,0,0,0,4.285714e-03,0.005714,0.016333,51.714286,54.357143,53.300000,62.142857,64.785714,63.333333,42.571429,46.714286,45.133333
3,6584,2019-03-09,3602,SE MILITARY DR,83,0.0,2019-03-09 15:37:00,2019-03-09 17:00:00,1.383333,Grease,Spill ContainedArea Cleaned and DisinfectedFlu...,NaN,SALADO CREEK,92804,92805,EASEMENT,None,3.0,3,2019,10,0,8.0,213.910000,PVC,1983.0,NaN,NaN,3602 SE MILITARY DR,1,1.0,0,GRAVITY,Sewer Main,NaN,33.0,2019-03-09 16:10:00,False,NaN,0.00,0.0,0.0,0.0,NaN,"(50, 250]","3602 SE MILITARY DR,SAN ANTONIO,Texas,USA","Quality Suites, Southeast Military Drive, Hill...",78223,36.0,"(35, 40]",01:23:00,0.03,0.0,0.0,74.0,87,60,1,0,0,0,0,0,4.285714e-03,0.005714,0.016333,51.714286,54.357143,53.300000,62.142857,64.785714,63.333333,42.571429,46.714286,45.133333
4,6580,2019-03-06,100,PANSY LN,75,0.0,2019-03-06 09:40:00,2019-03-06 09:55:00,0.250000,Structural,Spill ContainedArea Cleaned and DisinfectedFlu...,CLEANED MAIN,SALADO CREEK,61141,49543,STREET,None,2.0,3,2019,10,0,12.0,291.900000,CP,1952.0,NaN,NaN,100 PANSY LN,2,2.0,2018-12-15 00:00:00,GRAVITY,Sewer Main,NaN,0.0,2019-03-06 09:40:00,False,NaN,0.00,0.0,0.0,0.0,NaN,"(50, 250]","100 PANSY LN,SAN ANTONIO,Texas,USA","Pansy Lane, San Antonio, Bexar County, Texas, ...",78209,67.0,"

In [3]:
columns_to_drop_from_model = [
    "sso_id",
    "report_date",
    "spill_address_num",
    "spill_st_name",
    "spill_stop",
    "spill_start",
    "cause",
    "comments",
    "actions",
    "month",
    "year",
    "week",
    "spill_address_full",
    "last_cleaned",
    "response_dttm",
    "prevspill_24mos",
    "public_notice",
    "country_address",
    "location",
    "inches_no",
    "rainfall_last3",
    "unit_id",
    "unit_id2",
    "zip_code",
    "discharge_to",
    "discharge_route",
    "council_district",
    "time_spilled",
    "hrs",
    "gals_ret",
    "pipe_diam",
    "pipe_len",
    "response_time",
    "days_since_cleaned",
    "age"
]

df = df.drop(columns=columns_to_drop_from_model)
df

,total_gal,watershed,earz_zone,pipe_type,inst_year,num_spills_recorded,num_spills_24mos,unit_type,asset_type,root_cause,hrs_2,gal_2,hrs_3,gal_3,total_gal_binned,age_binned,precipitation,snowfall,snow_depth,avg_temp,max_temp,min_temp,foggy,heavy_fog,thunder,hail,blowing_snow,rain,precipitation_rolling_7,precipitation_rolling_14,precipitation_rolling_30,avg_temp_rolling_7,avg_temp_rolling_14,avg_temp_rolling_30,max_temp_rolling_7,max_temp_rolling_14,max_temp_rolling_30,min_temp_rolling_7,min_temp_rolling_14,min_temp_rolling_30
0,2100,SALADO CREEK,0,PVC,1997.0,1,1.0,GRAVITY,Sewer Main,NaN,0.00,0.0,0.0,0.0,"(1000, 5000]","(20, 25]",0.00,0.0,0.0,69.0,79,60,1,1,0,0,0,0,4.285714e-03,0.005714,0.016333,54.285714,55.285714,54.200000,65.857143,65.714286,64.533333,46.000000,47.642857,45.900000
1,80,DOS RIOS,0,PVC,1988.0,1,1.0,GRAVITY,Sewer Main,NaN,0.00,0.0,0.0,0.0,"(50, 250]","(30, 35]",0.00,0.0,0.0,69.0,79,60,1,1,0,0,0,0,4.285714e-03,0.005714,0.016333,54.285714,55.285714,54.200000,65.857143,65.714286,64.533333,46.000000,47.642857,45.900000
2,79,DOS RIOS,0,CP,1955.0,1,1.0,GRAVITY,Sewer Main,NaN,1.15,69.0,0.0,0.0,"(50, 250]","(60, 65]",0.03,0.0,0.0,74.0,87,60,1,0,0,0,0,0,4.285714e-03,0.005714,0.016333,51.714286,54.357143,53.300000,62.142857,64.785714,63.333333,42.571429,46.714286,45.133333
3,83,SALADO CREEK,0,PVC,1983.0,1,1.0,GRAVITY,Sewer Main,NaN,0.00,0.0,0.0,0.0,"(50, 250]","(35, 40]",0.03,0.0,0.0,74.0,87,60,1,0,0,0,0,0,4.285714e-03,0.005714,0.016333,51.714286,54.357143,53.300000,62.142857,64.785714,63.333333,42.571429,46.714286,45.133333
4,75,SALADO CREEK,0,CP,1952.0,2,2.0,GRAVITY,Sewer Main,NaN,0.00,0.0,0.0,0.0,"(50, 250]","(65, 70]",0.00,0.0,0.0,42.0,56,31,0,0,0,0,0,0,2.857143e-03,0.006429,0.015667,46.571429,52.357143,53.600000,55.142857,62.071429,62.866667,38.571429,45.000000,45.233333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3178,2000,SALADO CREEK,0,VCP,1966.0,1,0.0,GRAVITY,Sewer Main,structural,0.00,0.0,0.0,0.0,"(1000, 5000]","(40, 45]",0.20,0.0,0.0,56.5,70,43,1,0,0,1,0,1,2.857143e-02,0.015714,0.015000,56.500000,58.107143,55.783333,67.000000,70.071429,67.600000,46.000000,46.142857,43.966667
3179,800,LEON CREEK,0,VCP,1976.0,2,0.0,GRAVITY,Sewer Main,structural,0.00,0.0,0.0,0.0,"(250, 1000]","(30, 35]",0.00,0.0,0.0,41.5,46,37,1,0,0,1,0,1,2.478176e-18,0.002143,0.008333,56.214286,57.071429,55.683333,67.714286,68.642857,67.633333,44.714286,45.500000,43.733333
3180,400,LEON CREEK,0,VCP,1958.0,1,0.0,GRAVITY,Sewer Main,debris,0.00,0.0,0.0,0.0,"(250, 1000]","(50, 55]",0.00,0.0,0.0,41.5,46,37,1,0,0,1,0,1,2.478176e-18,0.002143,0.008333,56.214286,57.071429,55.683333,67.714286,68.642857,67.633333,44.714286,45.500000,43.733333
3181,25,SALADO CREEK,0,VCP,1972.0,1,0.0,GRAVITY,Sewer Main,structural,0.00,0.0,0.0,0.0,"(15, 50]","(35, 40]",0.00,0.0,0.0,41.5,46,37,1,0,0,1,0,1,2.478176e-18,0.002143,0.008333,56.214286,57.071429,55.683333,67.714286,68.642857,67.633333,44.714286,45.500000,43.733333


In [4]:
# fill null pipe type with unknown
# fill null root cause with unknown

df.pipe_type = df.pipe_type.fillna('Unknown')
df.root_cause = df.root_cause.fillna('Unknown')

In [5]:
categorical_columns = [
    "watershed",
    "earz_zone",
    "pipe_type",
    "inst_year",
    "unit_type",
    "asset_type",
    "age_binned",
    "total_gal_binned"
    
]

for column in categorical_columns:
    
    if is_numeric_dtype(df[f"{column}"]):
        values = df[f"{column}"].unique()

        for value in values:
            df[f"{column}_is_{value}"] = (df[f"{column}"] == value).astype(int)

        df = df.drop(columns=column)

    elif is_string_dtype(df[f"{column}"]):
        values = df[f"{column}"].astype(str).str.lower().unique()

        for value in values:
            df[f"{column}_is_{value}"] = (df[f"{column}"] == value).astype(int)

        df = df.drop(columns=column)

In [6]:
X = df.drop(columns='root_cause')
y = df.root_cause

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=13)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = .25, random_state=13)

In [7]:
tree = DecisionTreeClassifier(max_depth=5, random_state=13)
tree.fit(X_train, y_train)

print(f'Accuracy Score on Train: {tree.score(X_train, y_train) * 100:.2f}%')
print(f'Accuracy Score on Val: {tree.score(X_val, y_val) * 100:.2f}%')

Accuracy Score on Train: 51.34%
Accuracy Score on Val: 49.76%


In [8]:
preds = tree.predict(X_train)
print(classification_report(y_train, preds))

                   precision    recall  f1-score   support

          Unknown       0.00      0.00      0.00         5
by pass pump leak       0.00      0.00      0.00         2
       contractor       1.00      0.02      0.04        95
           debris       0.53      0.04      0.07       265
           grease       0.88      0.02      0.04       366
              i/i       0.00      0.00      0.00         4
     lift station       1.00      0.40      0.57        82
            other       1.00      0.06      0.11        35
       rain event       0.72      0.88      0.79       245
            roots       0.00      0.00      0.00        13
       structural       0.46      0.95      0.62       748
        vandalism       0.00      0.00      0.00        49

         accuracy                           0.51      1909
        macro avg       0.47      0.20      0.19      1909
     weighted avg       0.62      0.51      0.39      1909



/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
preds = tree.predict(X_val)
print(classification_report(y_val, preds))

                   precision    recall  f1-score   support

          Unknown       0.00      0.00      0.00         1
by pass pump leak       0.00      0.00      0.00         1
       contractor       0.00      0.00      0.00        36
           debris       0.20      0.01      0.02        83
           grease       1.00      0.02      0.03       122
              i/i       0.00      0.00      0.00         5
     lift station       0.80      0.25      0.38        32
            other       0.00      0.00      0.00        15
       rain event       0.74      0.83      0.78        81
            roots       0.00      0.00      0.00         5
       structural       0.45      0.97      0.62       246
        vandalism       0.00      0.00      0.00        10

         accuracy                           0.50       637
        macro avg       0.27      0.17      0.15       637
     weighted avg       0.53      0.50      0.37       637



/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
dot_data = export_graphviz(tree, 
                           feature_names= X_train.columns.tolist(),
                           class_names= y_train.value_counts(normalize=True).index.sort_values().tolist(),
                           out_file=None) 
graph = graphviz.Source(dot_data) 

graph.render('saws-prediction-tree', view=True)

'saws-prediction-tree.pdf'

In [11]:
train_preds = tree.predict(X_train)
val_preds = tree.predict(X_val)
train_comparison = pd.DataFrame({'training_actuals': y_train, 'training_preds': train_preds})
val_comparison = pd.DataFrame({'val_actuals': y_val, 'val_preds': val_preds})

In [12]:
train_comparison.to_csv('training_predictions.csv', index=False)
val_comparison.to_csv('validation_predictions.csv', index=False)

In [14]:
preprocessing.get_model_data()

NameError: name 'is_numeric_dtype' is not defined